# pybind11: binding between Python and C++

1. Wrapping API
2. Python objects, reference counting
3. Python containers

# Wrapping API

# Python objects and reference counting

# Python containers

# Exercises

# References